In [1]:
import pandas as pd 

class ExtratoLoader:
    def __init__(self, df_path):
        self.df_path = df_path
    def load_extrato(self):
        df = pd.read_excel(self.df_path)
        
        df.columns = df.iloc[4] 
        df = df.drop(df.index[:5])
        df.reset_index(drop=True, inplace=True) 
        df.columns = ([ 'Data', 'Descrição', 'Docto', 'Situação', 'Crédito (R$)', 'Débito (R$)', 'Saldo (R$)'])

        df = df.drop(columns=['Docto', 'Situação', 'Saldo (R$)'])
        df = df.fillna(0)

        df = df[df['Data'].astype(str).str.match(r'\d{2}/\d{2}/\d{4}', na=False)]
        
        df = df.drop(df.index[-1])
        df.reset_index(drop=True, inplace=True)

        # Função para converter números brasileiros para americanos
        def convert_br_to_float(value):
            if pd.isna(value) or value == 0:
                return 0.0
            # Converte para string e trata formato brasileiro
            str_val = str(value)
            # Remove pontos (separadores de milhares) e troca vírgula por ponto
            str_val = str_val.replace('.', '').replace(',', '.')
            try:
                return float(str_val)
            except:
                return 0.0

        # Converte colunas do formato brasileiro
        df['Crédito (R$)'] = df['Crédito (R$)'].apply(convert_br_to_float)
        df['Débito (R$)'] = df['Débito (R$)'].apply(convert_br_to_float)
        
        # Cria coluna Valor: Crédito como positivo, Débito como negativo
        df['Valor'] = df['Crédito (R$)'] + df['Débito (R$)']
        
        # Remove as colunas originais
        df = df.drop(columns=['Crédito (R$)', 'Débito (R$)'])

        return df

    def get_description_col(self, df):
        return list(df["Descrição"].values)

In [2]:
EL = ExtratoLoader(df_path=r'extratos\planilhaExtrato.xls')

df = EL.load_extrato()
df.head(10)

,Data,Descrição,Valor
0,07/08/2025,PIX ENVIADO Gabriel Lea...,-140.00
1,07/08/2025,COMPRA CARTAO DEB MC 07/08 DIVIN...,-26.00
2,07/08/2025,COMPRA CARTAO DEB MC 07/08 SAVEG...,-335.47
3,07/08/2025,COMPRA CARTAO DEB MC 07/08 DROGAVEN,-89.73
4,07/08/2025,COMPRA CARTAO DEB MC 07/08 FERNA...,-88.57
5,07/08/2025,COMPRA CARTAO DEB MC 07/08 FARMA...,-66.60
6,07/08/2025,COMPRA CARTAO DEB MC 07/08 POSTO...,-100.00
7,07/08/2025,PAGAMENTO DE BENEFICIOS DO INSS NB:19485896...,2113.79
8,06/08/2025,COMPRA CARTAO DEB MC 06/08 MINAS...,-19.83
9,05/08/2025,TED RECEBIDA BRADESCO SA...,153.66


In [3]:
desc = EL.get_description_col(df)
desc

['PIX ENVIADO                        Gabriel Leal Bonavina',
 'COMPRA CARTAO DEB MC               07/08 DIVINO GOURMET',
 'COMPRA CARTAO DEB MC               07/08 SAVEGNAGO LOJA 30 A',
 'COMPRA CARTAO DEB MC               07/08 DROGAVEN',
 'COMPRA CARTAO DEB MC               07/08 FERNANDO B FRUTAS',
 'COMPRA CARTAO DEB MC               07/08 FARMACIA SANTA PAUL',
 'COMPRA CARTAO DEB MC               07/08 POSTO ARCENAL LTDA',
 'PAGAMENTO DE BENEFICIOS DO INSS    NB:1948589602 07/2025',
 'COMPRA CARTAO DEB MC               06/08 MINAS QUEIJOS AVENI',
 'TED RECEBIDA                       BRADESCO SAUDE S A',
 'TRANSFERENCIA PROGRAMADA           PARA: 3432.60.004721-6',
 'PIX RECEBIDO                       Alvaro Roberto Bonavina',
 'COMPRA CARTAO DEB MC               02/08 DROGAVEN',
 'COMPRA CARTAO DEB MC               02/08 VITORIA PANIFIC',
 'COMPRA CARTAO DEB MC               02/08 FARMACIA A THERAPEU',
 'IOF ADICIONAL - AUTOMATICO         PERIODO: 01/07 A 31/07/25',
 'IOF IMPOSTO 

In [4]:
import google.generativeai as genai
from dotenv import load_dotenv, find_dotenv
import os


c:\Users\Gabriel\Documents\GitHub\dashboard-fatura\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import time

def classify_transaction_with_gemini_smart(transactions):
    load_dotenv(find_dotenv())
    
    # Configura API do Gemini
    genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
    model = genai.GenerativeModel('gemini-1.5-flash')
    
    batch_size = 15  
    all_categories = []
    
    for i in range(0, len(transactions), batch_size):
        batch = transactions[i:i+batch_size]
        batch_text = "\n".join([f"{idx+1}. {trans}" for idx, trans in enumerate(batch)])
        
        prompt = f"""
                Você é um especialista em classificação de transações financeiras de pessoa física.

                Analise a descrição da transação e classifique em UMA das categorias abaixo:
                Classifique EXATAMENTE {batch_size} transações abaixo:

                CATEGORIAS E CRITÉRIOS:
                - Alimentação: Restaurantes, lanchonetes, delivery, bares, cafeterias
                - Receitas: Salários, PIX recebidos, depósitos, rendimentos, reembolsos
                - Saúde: Consultas médicas, exames, planos de saúde, hospitais, clínicas
                - Farmácia: Drogarias, medicamentos (Raia, Droga Raia, Drogasil, etc.)
                - Seguros: Seguros de vida, auto, residencial, previdência
                - Mercado: Supermercados, hipermercados, açougues, padarias, hortifrúti, frutas, etc.
                - Educação: Escolas, cursos, livros, mensalidades, material escolar
                - Compras: Lojas de roupas, eletrônicos, casa, decoração, shopping
                - Transporte: Combustível, Uber, táxi, ônibus, metrô, estacionamento
                - Investimento: Aplicações, fundos, ações, renda fixa, corretoras
                - Transferências para terceiros: PIX enviados, DOC, TED para outras pessoas
                - Telefone: Operadoras de celular, internet, TV por assinatura
                - Moradia: Aluguel, condomínio, IPTU, luz, água, gás, reformas

                INSTRUÇÕES ESPECÍFICAS:
                - Raia, Drogaven = Farmácia
                - Supermercados (Savegnago, Casa Deliza, Braghini, etc.) = Mercado
                - Postos de gasolina = Transporte
                - Shopping/Magazine = Compras
                - PIX recebido = Receitas
                - PIX enviado = Transferências para terceiros
                - Aldeia, Divino Gourmet, = Alimentação

                Transação para classificar:
                {batch_text}

                Responda APENAS com o nome da categoria (sem pontuação ou explicação).
                """
        
        try:
            response = model.generate_content(prompt)
            batch_categories = response.text.strip().split('\n')
            
            cleaned_categories = []
            for cat in batch_categories:
                if cat.strip():
                    clean_cat = cat.split('. ', 1)[-1].strip()
                    if clean_cat:
                        cleaned_categories.append(clean_cat)
            
            if len(cleaned_categories) > len(batch):
                cleaned_categories = cleaned_categories[:len(batch)]
            elif len(cleaned_categories) < len(batch):
                cleaned_categories.extend(['Erro'] * (len(batch) - len(cleaned_categories)))
            
            all_categories.extend(cleaned_categories)
            print(f"Lote {i//batch_size + 1} processado - {len(cleaned_categories)} categorias")
            
            time.sleep(5)
            
        except Exception as e:
            if "429" in str(e): 
                print(f"Rate limit atingido! Aguardando 120 segundos...")
                time.sleep(120) 
                try:
                    response = model.generate_content(prompt)
                except Exception as e2:
                    print(f"Erro persistente: {e2}")
                    all_categories.extend(['Erro'] * len(batch))
            else:
                print(f"Erro: {e}")
                all_categories.extend(['Erro'] * len(batch))
    
    return all_categories

# Testa com batches maiores e rate limiting
catego = classify_transaction_with_gemini_smart(desc)
print(f"Total processado: {len(catego)} categorias")

Lote 1 processado - 15 categorias
Lote 2 processado - 15 categorias
Lote 3 processado - 15 categorias
Rate limit atingido! Aguardando 60 segundos...
Erro persistente: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 59
}
]
Rate limit atingido! Aguardando 60 segundos...


In [ ]:
catego

['Transferências para terceiros',
 'Alimentação',
 'Mercado',
 'Farmácia',
 'Mercado',
 'Farmácia',
 'Transporte',
 'Receitas',
 'Mercado',
 'Saúde',
 'Transferências para terceiros',
 'Receitas',
 'Farmácia',
 'Mercado',
 'Farmácia',
 'Investimentos',
 'Investimentos',
 'Transferências para terceiros',
 'Receitas',
 'Receitas',
 'Transferências para terceiros',
 'Transferências para terceiros',
 'Receitas',
 'Receitas',
 'Alimentação',
 'Mercado',
 'Mercado',
 'Mercado',
 'Mercado',
 'Farmácia',
 'Mercado',
 'Mercado',
 'Alimentação',
 'Mercado',
 'Transferências para terceiros',
 'Transferências para terceiros',
 'Transferências para terceiros',
 'Mercado',
 'Mercado',
 'Transferências para terceiros',
 'Moradia',
 'Receitas',
 'Moradia',
 'Moradia',
 'Investimentos',
 'Mercado',
 'Mercado',
 'Moradia',
 'Alimentação',
 'Alimentação',
 'Transferências para terceiros',
 'Transferências para terceiros',
 'Alimentação',
 'Receitas',
 'Receitas',
 'Mercado',
 'Compras',
 'Transferências 

In [ ]:
# def integrate_category_in_df(path_extrato):
#     EL = ExtratoLoader(path_extrato)
#     df = EL.load_extrato()
#     desc = EL.get_description_col(df=df)

#     categories = classify_transaction_with_gemini_smart(desc)

EL = ExtratoLoader(df_path=r'extratos\planilhaExtrato.xls')
df_clean = EL.load_extrato()

df_clean["Categoria"] = catego
df_clean.to_csv("finances.csv", index=False)